In [ ]:
# importing required libraries
import numpy as np
import pandas as pd

import joblib
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [2]:
test_df=pd.read_csv(r"C:\Users\arind\OneDrive\Desktop\Test NIDS\Training and Testing Sets\UNSW_NB15_testing-set.csv")

In [3]:
test_df.duplicated().sum()
test_df.drop_duplicates(inplace=True)

In [4]:
# Identify the data types of columns
column_data_types = test_df.dtypes

# Count the numerical and categorical columns
numerical_count = 0
categorical_count = 0

for column_name, data_type in column_data_types.items():
    if np.issubdtype(data_type, np.number):
        numerical_count += 1
    else:
        categorical_count += 1

print(f"There are {numerical_count} Numerical Columns in dataset")
print(f"There are {categorical_count} Categorical Columns in dataset")

There are 41 Numerical Columns in dataset
There are 4 Categorical Columns in dataset


In [5]:
# Rename columns in test_df as per train dataset
test_df = test_df.rename(columns={
    'smean': 'smeansz',
    'dmean': 'dmeansz',
    'dpkts': 'Dpkts',
    'sjit': 'Sjit',
    'djit': 'Djit'
})

# Verify
print(test_df.columns)

Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'Dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'Sjit', 'Djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smeansz', 'dmeansz', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')


In [ ]:
test_df.replace({'service': {'-': np.nan}}, inplace=True)

# Use KNN imputation for fill missing values in service column
service_encoder=joblib.load(r"C:\Users\arind\OneDrive\Desktop\Network IDS\models\service_encoder.pkl")
test_df['service'] = service_encoder.transform(test_df['service'].astype(str))

imputer = KNNImputer(n_neighbors=7)
test_df['service'] = imputer.fit_transform(test_df[['service']])

In [10]:
test_df['attack_cat'] = test_df['attack_cat'].apply(lambda x: x.strip().lower())

attack_cat_encoder = LabelEncoder()

test_df["attack_cat_encoded"] = attack_cat_encoder.fit_transform(test_df["attack_cat"])
attack_cat_mapping = dict(zip(attack_cat_encoder.classes_, attack_cat_encoder.transform(attack_cat_encoder.classes_)))
test_df.drop('attack_cat', axis=1, inplace=True)

In [12]:
proto_encoder=joblib.load(r"C:\Users\arind\OneDrive\Desktop\Network IDS\models\proto_encoder.pkl")
test_df["proto"] = proto_encoder.transform(test_df["proto"])

In [13]:
state_encoder = joblib.load(r"C:\Users\arind\OneDrive\Desktop\Network IDS\models\state_encoder.pkl")
test_df["state"] = state_encoder.transform(test_df["state"])

In [16]:
test_df.to_csv(r"C:\Users\arind\OneDrive\Desktop\Network IDS\data\processed\test_df_processed.csv", index=False)